In [2]:
import numpy as np
import warnings

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

In [3]:
warnings.filterwarnings('ignore')

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [ ]:
def read_file(try_file, except_file, sep):
    try:
        try:
            data_file = pd.read_csv(try_file, sep=sep)
        except:
            data_file = pd.read_csv(try_file)
    except:
        try:
            data_file = pd.read_csv(except_file, sep=sep)
        except:
            data_file = pd.read_csv(except_file)
    return data_file

geo_data_0 = read_file('YOUR_DATA', 'YOUR_DATA', '')
geo_data_1 = read_file('YOUR_DATA', 'YOUR_DATA', '')
geo_data_2 = read_file('YOUR_DATA', 'YOUR_DATA', '')

In [5]:
TEST_SIZE = 0.25
RANDOM_STATE = 42

In [6]:
display(geo_data_0)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [7]:
display(geo_data_1)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [8]:
display(geo_data_2)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [9]:
def df_info(df) -> pd.DataFrame:
    information = df.info()
    print()
    return information

df_info(geo_data_0)
df_info(geo_data_1)
df_info(geo_data_2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Nul

In [10]:
def df_describe(df) -> pd.DataFrame:
    disp = display(df.describe())
    return disp

df_describe(geo_data_0)
df_describe(geo_data_1)
df_describe(geo_data_2)

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [11]:
def df_nan(df) -> int:
    nans = display(df.isna().sum())
    return nans

df_nan(geo_data_0)
df_nan(geo_data_1)
df_nan(geo_data_2)

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [12]:
def df_duplicated(df) -> int:
    duplicates = print(df.duplicated().sum())
    return duplicates

df_duplicated(geo_data_0)
df_duplicated(geo_data_1)
df_duplicated(geo_data_2)

0
0
0


Данные геологоразведки трёх регионов находятся в датасетах:

    -geo_data_0
    -geo_data_1
    -geo_data_2
    
id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

## Обучение и проверка модели

In [13]:
geo_data_0 = geo_data_0.drop('id', axis=1)
geo_data_1 = geo_data_1.drop('id', axis=1)
geo_data_2 = geo_data_2.drop('id', axis=1)

In [14]:
def train_test(df, target):
    X = df.drop(target, axis=1)
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size=TEST_SIZE, 
                                                        random_state=RANDOM_STATE)
    return X_train, X_test, y_train, y_test

Написал функцию train_test чтобы разбить данные на выборки

In [15]:
X_train_0, X_test_0, y_train_0, y_test_0 = train_test(geo_data_0, 'product')
X_train_1, X_test_1, y_train_1, y_test_1 = train_test(geo_data_1, 'product')
X_train_2, X_test_2, y_train_2, y_test_2 = train_test(geo_data_2, 'product')

In [16]:
def model_fit(df, X_train, y_train, X_test, y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)  
    return predictions

функция model_fit обучает модель линейной регрессии

In [17]:
def print_metrics(df, y_test, predict):
    mse = mean_squared_error(y_test, predict)
    rmse = np.sqrt(mse)
    predicted_product_mean = np.mean(predict)
    print(f'Средний запас предсказанного сырья: {predicted_product_mean:.3f}')
    print(f'Средний запас сырья: {df["product"].mean():.3f}')
    print(f'RMSE = {rmse:.3f}')

функция print_metrics выводит предсказание модели, фактический запас сырья и метрику rmse

In [18]:
model_fit(geo_data_0, X_train_0, y_train_0, X_test_0, y_test_0)
print_metrics(geo_data_0, y_test_0, model_fit(geo_data_0, X_train_0, y_train_0, X_test_0, y_test_0))

Средний запас предсказанного сырья: 92.399
Средний запас сырья: 92.500
RMSE = 37.757


In [19]:
model_fit(geo_data_1, X_train_1, y_train_1, X_test_1, y_test_1)
print_metrics(geo_data_1, y_test_1, model_fit(geo_data_1, X_train_1, y_train_1, X_test_1, y_test_1))

Средний запас предсказанного сырья: 68.713
Средний запас сырья: 68.825
RMSE = 0.890


In [20]:
model_fit(geo_data_2, X_train_2, y_train_2, X_test_2, y_test_2)
print_metrics(geo_data_2, y_test_2, model_fit(geo_data_2, X_train_2, y_train_2, X_test_2, y_test_2))

Средний запас предсказанного сырья: 94.771
Средний запас сырья: 95.000
RMSE = 40.146


Лучше всего модель справилась со вторым регионом rmse равен 0.89

## Подготовка к расчёту прибыли

In [21]:
TOTAL_POINTS = 500 
BEST_POINT_FOR_SELECT = 200
BUDGET = 10_000_000_000
PRICE_FOR_BARREL = 450
INCOME_PER_PRODUCT = 450_000

обозначены константы для расчетов

In [22]:
break_even_well_volume = BUDGET / (BEST_POINT_FOR_SELECT * INCOME_PER_PRODUCT)
print(f'Объём сырья для безубыточной разработки новой скважины: {break_even_well_volume:.3f}')

Объём сырья для безубыточной разработки новой скважины: 111.111


расчитан объем сырья для безубыточной разработки скважины он равен 111 тыс. баррелей

In [23]:
def comparison(df, column, num_reg):
    diff = df[column].mean() - 111.111
    print(f'Средний запас в регионе {num_reg}: {df[column].mean():.3f}')
    print(f'Разница среднего запаса в {num_reg} регионе с безубыточной разработкой скважины равна {diff:.3f}')

In [24]:
comparison(geo_data_0, 'product', 1)

Средний запас в регионе 1: 92.500
Разница среднего запаса в 1 регионе с безубыточной разработкой скважины равна -18.611


In [25]:
comparison(geo_data_1, 'product', 2)

Средний запас в регионе 2: 68.825
Разница среднего запаса в 2 регионе с безубыточной разработкой скважины равна -42.286


In [26]:
comparison(geo_data_2, 'product', 3)

Средний запас в регионе 3: 95.000
Разница среднего запаса в 3 регионе с безубыточной разработкой скважины равна -16.111


Выведен средние запасы в регионах и сравнение с безубыточной разработкой скважины, больше всего запасов в третьем регионе

## Расчёт прибыли и рисков 

In [27]:
predictions_0 = model_fit(geo_data_0, X_train_0, y_train_0, X_test_0, y_test_0)
predictions_1 = model_fit(geo_data_1, X_train_1, y_train_1, X_test_1, y_test_1)
predictions_2 = model_fit(geo_data_2, X_train_2, y_train_2, X_test_2, y_test_2)

In [28]:
def calculate_income(target, predict, income_per_product, budget):
    target = pd.Series(target).reset_index(drop=True)
    predict = pd.Series(predict).reset_index(drop=True)
    sort_pred = predict.sort_values(ascending=False)
    best_wells = target[sort_pred.index][:200]
    total_income = income_per_product * best_wells.sum() - budget
    return total_income.round(3)

функция calculate_income расчитывает прибыль 200 лучишх скважин

In [29]:
def print_income(income, num_reg):
    print(f'Прибыль от 200 скважин {num_reg} региона с максимальным значением предсказания: {income} рублей')

In [30]:
print_income(calculate_income(y_test_0, predictions_0, INCOME_PER_PRODUCT, BUDGET), 'первого')

Прибыль от 200 скважин первого региона с максимальным значением предсказания: 3359141114.462 рублей


In [31]:
print_income(calculate_income(y_test_1, predictions_1, INCOME_PER_PRODUCT, BUDGET), 'второго')

Прибыль от 200 скважин второго региона с максимальным значением предсказания: 2415086696.682 рублей


In [32]:
print_income(calculate_income(y_test_2, predictions_2, INCOME_PER_PRODUCT, BUDGET), 'третьего')

Прибыль от 200 скважин третьего региона с максимальным значением предсказания: 2598571759.374 рублей


прибыль от 200 лучших скважин в регионах:

    Первый регион - 3.335 млрд. рублей
    Второй регион - 2.415 млрд. рублей
    Третий регион - 2.598 млрд. рублей

In [33]:
def bootstrap(target, predict, best_point_for_select, income_per_product, budget):
    state = RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.reset_index(drop=True).sample(n=500,
                                         replace=True,
                                         random_state=state)
        prob_subsample = predict[target_subsample.index]
        values.append(calculate_income(target_subsample, 
                                       prob_subsample, 
                                       income_per_product,
                                       budget))
        
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    loss_risk = (len(values[values < 0])/len(values))*100
    return lower, upper, mean, loss_risk

функция применяет технику bootstrap с 1000 выборок и находит распределение прибыли

In [34]:
lower_0, upper_0, mean_0, loss_risk_0 = bootstrap(y_test_0, 
                                                  predictions_0,
                                                  BEST_POINT_FOR_SELECT,
                                                  INCOME_PER_PRODUCT,
                                                  BUDGET)
lower_1, upper_1, mean_1, loss_risk_1 = bootstrap(y_test_1, 
                                                  predictions_1,
                                                  BEST_POINT_FOR_SELECT,
                                                  INCOME_PER_PRODUCT,
                                                  BUDGET)
lower_2, upper_2, mean_2, loss_risk_2 = bootstrap(y_test_2, 
                                                  predictions_2,
                                                  BEST_POINT_FOR_SELECT,
                                                  INCOME_PER_PRODUCT,
                                                  BUDGET)

In [35]:
print(f'Средняя прибыль первого региона составляет {mean_0} рублей')
print(f'Средняя прибыль второго региона составляет {mean_1} рублей')
print(f'Средняя прибыль третьего региона составляет {mean_2} рублей')

Средняя прибыль первого региона составляет 406278783.42441505 рублей
Средняя прибыль второго региона составляет 432624131.81312895 рублей
Средняя прибыль третьего региона составляет 377362192.42291796 рублей


In [36]:
print(f'95% доверительный интервал средней прибыли первого региона: {round(lower_0, 3)} {round(upper_0, 3)}')
print(f'95% доверительный интервал средней прибыли второго региона: {round(lower_1, 3)} {round(upper_1, 3)}')
print(f'95% доверительный интервал средней прибыли третьего региона: {round(lower_2, 3)} {round(upper_2, 3)}')

95% доверительный интервал средней прибыли первого региона: -117742136.495 911737050.752
95% доверительный интервал средней прибыли второго региона: 16846174.932 815972526.285
95% доверительный интервал средней прибыли третьего региона: -170780417.705 901772131.386


In [37]:
print(f'Риск убытков для первого региона равен {loss_risk_0} %')
print(f'Риск убытков для второго региона равен {loss_risk_1} %')
print(f'Риск убытков для третьего региона равен {loss_risk_2} %')

Риск убытков для первого региона равен 6.7 %
Риск убытков для второго региона равен 1.9 %
Риск убытков для третьего региона равен 7.3999999999999995 %


## Вывод

    Во время проекта 'Выбор локации для скважины', данные были обработаны: пропуски не 
    найдены, дубликаты не обнаружены, типы данных соответствуют самим данным. 

    По итогам можно сделать следующие выводы:

    -Данные были разделены на выборки, предсказания модели были сохранены
    rmse трех регионов:
        Первого - 37.357
        Второго - 0.890
        Третьего - 40.146
        
    -Значения для расчетов были сохранены в переменных константах и расчитан 
    объем для безубыточной разработки новой скважины - 111 тыс. баррелей
    
    -Написана функция для расчета прибыли по 200 лучшим скважинам и была 
    расчитана предсказана прибыль:
        Первый регион - 3.335 млрд. рублей
        Второй регион - 2.415 млрд. рублей
        Третий регион - 2.598 млрд. рублей
        
    -Посчитаны риск убытков, средняя прибыль и доверительный интервал 
    для всех регионов:
        Средняя прибыль:
            Первого региона - 406 млн. рублей
            Второго региона - 432 млн. рублей
            Третьего региона - 377 млн. рублей
        
        Доверительный интервал:
            Первого региона: -117742136.495 911737050.752
            Второго региона: 16846174.932 815972526.285
            Третьего региона: -170780417.705 901772131.386
            
        Риск убытков:
            Первого региона - 6.7%
            Второго региона - 1.9%
            Третьего региона - 7.39%
    
    По этим данным, можно сделать вывод, что для разработки скважин больше всего подходит второй регион